<b>DMXe though Jupyter Notebooks</b>

First step is to install the DMXe/DMXi package from nuget.

In [1]:
#r "nuget:DMXiServer"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package DMXiServer version 0.5.13

A connection to this package needs to be created to run queries against DMXe/DMXi.

In [2]:
#pragma warning disable CS1702  
using aconcagua.server;
var server = new AconcaguaService(null);

Once the connection is established, you can execute code queries.

This query will test the basic connection and check the serer version.

In [5]:
var getVersionResponse = server.GetVersion(null, null).Result;
Console.WriteLine(getVersionResponse.Version);

0.5.13


Here we will query the list of databases at the root of the server.

The server will use C:/Temp as the base directory for databases.

In [6]:
var getDatabasesResponse = server.GetDatabases(null, null).Result;

foreach(var db in getDatabasesResponse.Sourcenames)
{
    Console.WriteLine(db);
}

dmxe,brb_hist.dmxe
dmxe,brb_macrofw.20201214024043.dmxe
dmxe,brb_macrofw.dmxe
dmxe,JupyterTesting.dmxe
ecos,ECDATA_MFS


We can now select a database and query for the available series.

In [7]:
var getSeriesKeysRequest = new GetSeriesKeysRequest();
getSeriesKeysRequest.Sourcenames.Add("dmxe://localhost:5001/brb_hist.dmxe");
var getSeriesKeysResponse = server.GetSeriesKeys(getSeriesKeysRequest, null).Result;

foreach(var key in getSeriesKeysResponse.Keys)
{
    Console.WriteLine(key.Seriesname);
}

316BCA_N
316BGS_N
316BXG_N
316BMG_N
316BXG_FOB_N
316BXGMA_N
316BXGT_N
316BXGO_N
316BXG_BOP_N
316BXGM_N
316BXGM_SUG_N
316BXGM_ELEC_N
316BXGM_CHEM_N
316BXGM_FOODBEV_N
316BXGM_OTH_N
316BXGM_OTH_LIME_N
316BXGM_OTH_STRUCT_N
316BXGM_OTH_METAL_N
316BXGM_OTH_PRINT_N
316BXGM_OTH_PLASTIC_N
316BXGM_OTH_ORTHO_N
316BXGM_OTH_MACH_N
316BXGC_N
316BXGP_N
316BXGR_N
316BXG_MERCH_N
316BMG_CIF_N
316BMG_CIF_ADJ_N
316BMG_CIF_ADJ_FREIGHT_N
316BMG_CIF_ADJ_OTH_N
316BMG_BOP_N
316BMGM_N
316BMGMCON_N
316BMGMK_N
316BMGMI_N
316BMGMI_Fuel_N
316BMGM_OTH_N
316BMGC_N
316BMGP_N
316BMGR_N
316BS_N
316BSX_N
316BSM_N
316BSTV_N
316BXSTV_N
316BMSTV_N
316BSOTHER_N
316BXSOTHER_N
316BMSOTHER_N
316BSTR_N
316BXSTR_N
316BMSTR_N
316BSGS_N
316BXSGS_N
316BMSGS_N
316BI_N
316BIC_N
316BID_N
316BTR_N
316BTRC_N
316BTRCG_N
316BTRCO_N
316BTRD_N
316BTRDG_N
316BTRDGO_N
316BK_N
316BKTR_N
316BKTRG_N
316BKTRO_N
316BKDA_N
316BF_N
316BFD_N
316BFDL_N
316BFDA_N
316BFOTH_N
316BKF_LT_N
316BFG_N
316BFGL_N
316BFGL_DIS_N
316BFGLD_IMF_N
316BFGLD_AMT_N
316BF

Knowing the series name, we can query the metadata associated with the series.

In [8]:
var getMetadataRequest = new GetMetadataRequest();
getMetadataRequest.Keys.Add(new SourceSeriesKey
{
    Sourcename = "dmxe://localhost:5001/brb_macrofw.dmxe", Seriesname = "316NGDP"
});
getMetadataRequest.Metadataheaders.Add(new List<string> { "source", "scale", "unit", "2001A1"});
var getMetadataResponse = server.GetMetadata(getMetadataRequest, null).Result;

Console.WriteLine(getMetadataResponse.Metadataheaders);
foreach(var seriesData in getMetadataResponse.Seriesdata)
{
    Console.WriteLine(seriesData.Key.Seriesname);
    Console.WriteLine(seriesData.Values);
}

[ "source", "scale", "unit", "2001A1" ]
316NGDP
[ "IMF Staff Calculation", "Million", "", "5937.31219587926" ]


We can query for all observations of a certian frequency instead of individual headers.

In [10]:
public static class SourceSeriesKeyList
{
    public static IEnumerable<SourceSeriesKey> Create(string sourceName, IEnumerable<string> _seriesList)
    {
        return _seriesList.Select(s => new SourceSeriesKey() { Sourcename = sourceName, Seriesname = s }).ToList();
    }
}

var getObservationsRequest = new GetObservationsRequest();
getObservationsRequest.Keys.Add(
                SourceSeriesKeyList.Create(
                    "dmxe://localhost:5001/brb_macrofw.dmxe", new[] {"316NGDP", "316NGDP_YOY" }));
    
getObservationsRequest.Frequencies = "A";
var getObservationsResponse = server.GetObservations(getObservationsRequest, null).Result;

foreach(var seriesdata in getObservationsResponse.Seriesdata)
{
    Console.WriteLine($"{seriesdata.Key.Sourcename}\t {seriesdata.Key.Seriesname}: {seriesdata.Values.Count} observations");
}

dmxe://localhost:5001/brb_macrofw.dmxe	 316NGDP: 42 observations
dmxe://localhost:5001/brb_macrofw.dmxe	 316NGDP_YOY: 41 observations


We can utilize a visualization library to create a chart from this obervation data.

In [12]:
using XPlot.Plotly;

var xValues = new List<double>();
var yValues = new List<double>();
foreach(var keyValue in getObservationsResponse.Seriesdata[0].Values)
{
    yValues.Add(keyValue.Value);
    xValues.Add(double.Parse(keyValue.Key.Substring(0,4)));
}
    
Chart.Plot(
    new Graph.Scatter
    {
        x = xValues,
        y = yValues
    }
)

We are not limited to just query data, we can utilize DMXe save functionality.

In [13]:
var saveMetadataRequest = new SaveMetadataRequest();
saveMetadataRequest.Metadataheaders.Add(new List<string>{"2001A1", "2001Q1", "2001M1"});
saveMetadataRequest.Seriesdata.Add(new MetadataList
{
    Key = new SourceSeriesKey
    {
        Sourcename = "dmxe://localhost:5001/JupyterTesting.dmxe", Seriesname = "111a1"
    },
    Values = {"1", "2", "3"}
});

var saveMetadataResponse = server.SaveMetadata(saveMetadataRequest, null).Result;
Console.WriteLine(saveMetadataResponse);

{ "responsemetadata": { "111a1": "{\"Database\":\"C:\\\\Temp\\\\JupyterTesting.dmxe\",\"SeriesCode\":\"111a1\",\"Status\":\"NoChange\",\"Summary\":\"NoChange\",\"IndirectUpdate\":\"false\",\"TransactionId\":0}" } }


In [ ]:
var batchRequest = new BatchRequest {Iterations = 10, Convergence = true, Haltonerror = false};
batchRequest.Batchrequests.AddRange(new List<BatchAction>
{
    new BatchAction
    {
        Workbook = "DMXeSheet.xlsx", Worksheet = "Save", Execute = true, Action = "DMXe Save", Status = "", Lastexecuted = ""
    },
    new BatchAction
    {
        Workbook = "DMXeSheet.xlsx", Worksheet = "Refresh", Execute = true, Action = "DMXe Refresh", Status = "", Lastexecuted = ""
    }
});
var fileData = System.IO.File.ReadAllBytes(@"C:\Users\kfrolov\OneDrive - International Monetary Fund (PRD)\ServerBatch\DMXeSheet.xlsx");
batchRequest.Workbooks.Add("DMXeSheet.xlsx", Google.Protobuf.ByteString.CopyFrom(fileData));
var batchResponse = server.RunBatch(batchRequest, null).Result;
//Console.WriteLine(batchResponse.Responsemetadata["IterationsCompleted"]);